# Online Training

A variation of SBI is Sequential Neural Posterior Estimation (SNPE), where the model is trained online, i.e., in multiple rounds. In each round, simulations are generated from the current posterior estimate, and the model is updated with these new simulations. This approach can be more efficient in terms of simulation budget, especially when the prior is broad and the posterior is narrow.

However, the model is no longer amortized, as it is specialized to the specific observation after training. This means that a new model must be trained for each new observation, which can be computationally expensive if many observations need to be analyzed.

In [1]:
from synference import SBI_Fitter

fitter = SBI_Fitter.init_from_hdf5(model_name="test",
                                   hdf5_path="../example_grids/test_model_grid.hdf5")

fitter.create_feature_array();

2025-11-05 10:58:39,457 | synference | INFO     | ---------------------------------------------
2025-11-05 10:58:39,458 | synference | INFO     | Features: 8 features over 100 samples
2025-11-05 10:58:39,458 | synference | INFO     | ---------------------------------------------
2025-11-05 10:58:39,458 | synference | INFO     | Feature: Min - Max
2025-11-05 10:58:39,459 | synference | INFO     | ---------------------------------------------
2025-11-05 10:58:39,459 | synference | INFO     | JWST/NIRCam.F070W: 7.131974 - 42.758 AB
2025-11-05 10:58:39,459 | synference | INFO     | JWST/NIRCam.F090W: 7.108530 - 39.933 AB
2025-11-05 10:58:39,459 | synference | INFO     | JWST/NIRCam.F115W: 7.012560 - 38.354 AB
2025-11-05 10:58:39,460 | synference | INFO     | JWST/NIRCam.F150W: 6.969396 - 36.997 AB
2025-11-05 10:58:39,460 | synference | INFO     | JWST/NIRCam.F200W: 7.133157 - 35.470 AB
2025-11-05 10:58:39,460 | synference | INFO     | JWST/NIRCam.F277W: 7.670149 - 33.243 AB
2025-11-05 10:5

Now we will recreate the simulator from the grid data stored in the HDF5 file.

In [2]:
fitter.recreate_simulator_from_grid()

params: {'fesc': 'fesc', 'fesc_ly_alpha': 'fesc_ly_alpha', 'dust_curve': <synthesizer.emission_models.transformers.dust_attenuation.PowerLaw object at 0x106691b40>}
2025-11-05 10:58:40,335 | synference | INFO     | Simulator recreated from grid at ../example_grids/test_model_grid.hdf5.


GalaxySimulator(<class 'synthesizer.parametric.sf_hist.LogNormal'>,
                                        <class 'synthesizer.parametric.metal_dist.DeltaConstant'>,
                                        +----------------------------------------------------------------------------------------------------------+
|                                                   GRID                                                   |
+-----------------------------+----------------------------------------------------------------------------+
| Attribute                   | Value                                                                      |
+-----------------------------+----------------------------------------------------------------------------+
| grid_dir                    | '/Users/user/Library/Application Support/Synthesizer/grids'                |
+-----------------------------+----------------------------------------------------------------------------+
| grid_name                   

Now we can choose an observation for our multiple rounds of online training. Here, we will randomly select one of the simulations from our grid as the observation.

In [3]:
index = 20
sample = fitter.feature_array[index]
true_params = fitter.fitted_parameter_array[index]

sample

array([28.922905, 27.99003 , 27.05276 , 26.493101, 25.965355, 25.390919,
       24.99644 , 24.839458], dtype=float32)

Now we can run our online SBI model - to do this we set 'learning_type' to 'online', specify the number of online rounds with 'num_online_rounds', and provide our chosen observation with 'online_training_xobs'. We also set the number of simulations per round with 'num_simulations'. The engine is set to 'SNPE' to use Sequential Neural Posterior Estimation, but SNLE and SNRE are also available for online training.

In [8]:
fitter.run_single_sbi(
    online_training_xobs=sample,
    learning_type="online",
    engine="SNPE",
    num_simulations=10_000,
    num_online_rounds=3,
    override_prior_ranges={'peak_age':(10, 1000)}
)

2025-11-05 11:02:17,451 | synference | INFO     | ---------------------------------------------
2025-11-05 11:02:17,452 | synference | INFO     | Prior ranges:
2025-11-05 11:02:17,452 | synference | INFO     | ---------------------------------------------
2025-11-05 11:02:17,452 | synference | INFO     | redshift: 0.00 - 4.98 [dimensionless]
2025-11-05 11:02:17,454 | synference | INFO     | log_mass: 8.01 - 11.99 [log10_Msun]
2025-11-05 11:02:17,455 | synference | INFO     | tau_v: 0.01 - 3.00 [mag]
2025-11-05 11:02:17,456 | synference | INFO     | tau: 0.11 - 1.98 [dimensionless]
2025-11-05 11:02:17,457 | synference | INFO     | peak_age: 10.00 - 1000.00 [Myr]
2025-11-05 11:02:17,457 | synference | INFO     | log10metallicity: -3.98 - -1.41 [log10(Zmet)]
2025-11-05 11:02:17,464 | synference | INFO     | ---------------------------------------------
2025-11-05 11:02:17,482 | synference | INFO     | Processing prior...
2025-11-05 11:02:17,493 | synference | INFO     | Using provided xob

INFO:root:MODEL INFERENCE CLASS: SNPE
INFO:root:The first round of inference will simulate from the given proposal or prior.


Running 10000 simulations.:   0%|          | 0/10000 [00:00<?, ?it/s]

['tau', 'peak_age', 'max_age'] ['min_age']
{'tau': 1.2137778, 'peak_age': unyt_quantity(670.90234, dtype=float32, units='Myr'), 'max_age': unyt_quantity(1873.44316206, 'Myr')}
{'min_age': unyt_array(0, 'yr')}
['tau', 'peak_age', 'max_age'] ['min_age']
{'tau': 0.67164135, 'peak_age': unyt_quantity(263.068, dtype=float32, units='Myr'), 'max_age': unyt_quantity(3515.14310379, 'Myr')}
{'min_age': unyt_array(0, 'yr')}
['tau', 'peak_age', 'max_age'] ['min_age']
{'tau': 0.34091142, 'peak_age': unyt_quantity(45.04627, dtype=float32, units='Myr'), 'max_age': unyt_quantity(1256.7085715, 'Myr')}
{'min_age': unyt_array(0, 'yr')}
['tau', 'peak_age', 'max_age'] ['min_age']
{'tau': 1.7844267, 'peak_age': unyt_quantity(535.05164, dtype=float32, units='Myr'), 'max_age': unyt_quantity(11664.71797735, 'Myr')}
{'min_age': unyt_array(0, 'yr')}
['tau', 'peak_age', 'max_age'] ['min_age']
{'tau': 1.2906495, 'peak_age': unyt_quantity(652.2978, dtype=float32, units='Myr'), 'max_age': unyt_quantity(1340.79073548

KeyboardInterrupt: 